# **PreProcess**
테이블 자료간 연결을 위한 전처리 및 Index 작업

In [2]:
from glob import glob
file_list = glob("data/food_*.x")
file_list

[]

# **레시피 데이터**
## **1 레시피 데이터 전처리**
데이터 프레임 자료 불러오기

In [2]:
import pandas as pd
food_menu_org = pd.read_excel(file_list[1])
food_menu_org.shape, food_menu_org.head(2)

((1198, 54),
    소유자         메뉴명 조리방법 요리종류  중량(1인분)     열량  탄수화물   단백질   지방    나트륨  ...  \
 0  식약처  칼륨 듬뿍 고구마죽  끓이기   후식      NaN  205.0  35.0   3.0  6.0   68.0  ...   
 1  식약처  누룽지 두부 계란죽  끓이기    밥      NaN  380.0  67.0  12.0  7.0  271.0  ...   
 
   만드는법_16 만드는법_이미지_16 만드는법_17 만드는법_이미지_17 만드는법_18 만드는법_이미지_18 만드는법_19  \
 0     NaN         NaN     NaN         NaN     NaN         NaN     NaN   
 1     NaN         NaN     NaN         NaN     NaN         NaN     NaN   
 
   만드는법_이미지_19 만드는법_20 만드는법_이미지_20  
 0         NaN     NaN         NaN  
 1         NaN     NaN         NaN  
 
 [2 rows x 54 columns])

## **2 개별 레시피 Json 구분**
레시피 데이터 Json 재료와 전처리
1. 중분류 Tag 가 포함된 것도 있지만, 대부분은 재료만 포함
1. 우선 개별 list 객체로 세분화 하기
1. 추후에 중간묶음, 개별 재료와 단위명 구분하기
1. 이 작업은 블로그등 다양한 

In [3]:
food_menu = food_menu_org.loc[:,["소유자","메뉴명","재료정보"]]
food_menu["재료정보"] = [str(_).splitlines() for _ in food_menu["재료정보"]]
food_menu.head(3)

,소유자,메뉴명,재료정보
0,식약처,칼륨 듬뿍 고구마죽,"[고구마죽, 고구마 100g(2/3개), 설탕 2g(1/3작은술), 찹쌀가루 3g(..."
1,식약처,누룽지 두부 계란죽,"[채소준비, 애호박 30g(1/6개), 표고버섯 20g(2개), 당근 5g(3×2×..."
2,식약처,오색지라시 스시,"[초밥, 밥 210g(1공기), 배합초, 식초 20g(1⅓큰술), 설탕 10g(2작..."


In [4]:
food_menu['재료정보'][2]

['초밥',
 '밥 210g(1공기)',
 '배합초',
 '식초 20g(1⅓큰술), 설탕 10g(2작은술), 소금 약간',
 '오색지라시',
 '달걀노른자 40g(달걀2개), 표고버섯 10g(1장), 새송이버섯 20g(3개), 새우 25g(4마리), 홍피망 10g(1/8개), 청피망 10g(1/8개), 양파 5g(5×1cm), 대두유 20g(1⅓큰술), 참기름 10g(2작은술), 식용유 약간']

## **3 Detail 구분하기**
레시피 데이터 Json 재료와 전처리
```
tag_item = re.findall(r"\(.*?\)", temp)[0]
```

In [30]:
recipe_temp = food_menu['재료정보'][5]
token_temp  = recipe_temp[0]
token_temp  = "참깨 5g(조금),  소금 조금"
token_temp.split(',')

['참깨 5g(조금)', '  소금 조금']

In [31]:
import re
re.findall(r"\w+", token_temp)

['참깨', '5g', '조금', '소금', '조금']

In [32]:
re.findall(r"\(.*?\)", token_temp)

['(조금)']

In [33]:
re.findall(r"\d+g", token_temp)

['5g']

In [34]:
import re
re.findall(r"(조금|약간)", token_temp)

['조금', '조금']

In [35]:
re.findall(r"\d+g", token_temp)
token_temp.split('(')[0]

'참깨 5g'

## **4 메뉴명 공통분모 적용해보기**
기존에 작업한 기준의 성능 평가하기

In [1]:
with open("data/nouns_tokens.txt" , "r") as f:
    nouns_token = f.read()
nouns_token = nouns_token.split(",")
len(nouns_token)

686

In [46]:
menu_org = list(food_menu_org.메뉴명)

In [52]:
from muyong.nlp import food_nouns
menu_valid = [["_".join(food_nouns(_, nouns_token)), _]  for _ in menu_org]

In [56]:
import pandas as pd
menu_names = pd.DataFrame(menu_valid)

In [58]:
menu_names.to_csv("menu_tokens.csv")

In [59]:
# Plus Token
스시, 영양밥, 관자요리, 청국장, 고소, 담백, 파르페 ,자장면, 돌나물, 전복, 화이타, 아보카도
카나페, 크런치롤, 구기자, 월남쌈, 오코노미야키, 크러스트, 크로켓, 우둔살, 스틱, 수란, 삼합
카프레제, 인삼, 초계, 백김치 ,토란, 배숙, 누룽, 크로켓, 차우더, 부각, 롤, 전병, 황태포, 면, 월남쌈, 리조또, 아란치니,
육회, 대하, 삼계, 약식, 카나페, 쇠고기, 해산물, 차돌박이, 대하, 양갱, 게살, 딤섬,무스, 수정과, 아스파라거스 ,크로켓,
비지, 나가사키,  (361 계속작업)

NameError: name '스시' is not defined

In [ ]:
# More Unique (나누기)
들깨소스, 비트, 가자미조림

# **레시피 데이터 작업용 함수**
## **1 레시피 데이터 전처리**
단일한 기준으로 자료 정리하기
1. **메뉴명** 과 **식재료** 기준 정하기
1. **어떤 작업을 해야할 지 애매해서** 방황하는 부분이 많았음!!!
1. **만개의 레시피** 등과 같은 자료를 통해 **공통분모** 추출하기

In [9]:
from muyong.nlp import food_nouns
# food_nouns()

In [10]:
from glob import glob
file_list = glob("data/food_*.xls")
file_list

['data/food_barcode_info.xls',
 'data/food_recipe_info.xls',
 'data/food_barcode.xls',
 'data/food_product.xls',
 'data/food_nutrient.xls',
 'data/food_recall.xls',
 'data/food_Garak.xls']

In [15]:
import pandas as pd
food_ingre_org = pd.read_excel(file_list[4])
food_ingre_org.shape, food_ingre_org.head(2)

((13824, 17),
    번호                식품코드        식품군    식품이름  ...  트랜스지방(g)(1회제공량당)  가공업체명  구축년도              자료원
 0   1  100101000100000001  곡류 및 그 제품  고량미,알곡  ...               NaN    NaN  2001  농촌진흥청  식품성분표 DB
 1   2  100101000200200001  곡류 및 그 제품  겉귀리,생것  ...               NaN    NaN  2017  농촌진흥청  식품성분표 DB
 
 [2 rows x 17 columns])

In [18]:
len(set(food_ingre_org.식품이름))

13088

In [20]:
list(food_ingre_org.식품이름)

['고량미,알곡',
 '겉귀리,생것',
 '쌀귀리,생것',
 '오트밀',
 '기장,생것',
 '메밀,생것',
 '메밀가루',
 '메밀국수,생면',
 '메밀국수,생면,삶은것',
 '메밀국수,건면',
 '메밀국수,건면,삶은것',
 '메밀가공(면류),메밀냉면,인스턴트,마른것',
 '통밀',
 '밀쌀',
 '강력밀가루',
 '중력밀가루',
 '박력밀가루',
 '당근케이크가루',
 '통밀가루',
 '도우넛가루',
 '부침가루',
 '빵가루',
 '밀가공(가루),튀김가루',
 '밀가공(가루),팬케이크가루',
 '국수',
 '국수,삶은것',
 '소면,건면',
 '소면,건면,삶은것',
 '라면,건면',
 '라면,조리',
 '진라면매운맛',
 '마카로니,건면',
 '마카로니,건면,삶은것',
 '스파게티,건면',
 '스파게티,건면,삶은것',
 '우동',
 '우동,생면',
 '우동,생면,삶은것',
 '냄비우동,냉동',
 '울면',
 '중국국수,생면',
 '중국국수,생면,삶은것',
 '중국국수,증숙면',
 '중국국수',
 '중국국수,삶은것',
 '쫄면',
 '칼국수,생면',
 '칼국수,반건면',
 '칼국수,건조',
 '건빵',
 '건포도빵',
 '곰보빵',
 '도우넛,링',
 '도우넛,이스트',
 '도우넛,케이크',
 '도우넛,팥',
 '소프트롤빵',
 '하드롤빵',
 '마늘빵',
 '만주,밤',
 '우유머핀',
 '잉글리쉬머핀',
 '밀가공(빵류),모카빵',
 '달걀베이글',
 '식빵',
 '옥수수식빵',
 '우유식빵',
 '토스트식빵',
 '옥수수빵',
 '잼빵',
 '크로아상',
 '크로아상',
 '버터크로아상',
 '밀가공(빵류),크림빵',
 '버터크림빵',
 '밀가공(빵류),팥빵',
 '패이스트리',
 '과일패이스트리',
 '치즈패이스트리',
 '고기호빵',
 '팥찐빵',
 '호밀빵',
 '초코파이',
 '슈크림빵',
 '와플',
 '와플잼',
 '와플커스터드크림',
 '팬케이크',
 '카스텔라',
 '감카스텔라',
 '달걀커스타드',
 '꽈배기',
 '모닝빵',
 '바게

In [21]:
food_ingre_org.columns

Index(['번호', '식품코드', '식품군', '식품이름', '1회제공량(g)', '열량(kcal)(1회제공량당)',
       '탄수화물(g)(1회제공량당)', '단백질(g)(1회제공량당)', '지방(g)(1회제공량당)', '당류(g)(1회제공량당)',
       '나트륨(mg)(1회제공량당)', '콜레스테롤(mg)(1회제공량당)', '포화지방산(g)(1회제공량당)',
       '트랜스지방(g)(1회제공량당)', '가공업체명', '구축년도', '자료원'],
      dtype='object')

In [1]:
test = {
    0:"python",
    1:"tensorflow"
}

In [3]:
test.get(4, 0)

0

In [4]:
def double(x):
    return x * 2

In [6]:
list(map(double, [1,2,3,4]))

[2, 4, 6, 8]

In [6]:
import re
def menu_recipe(menu1):
    #menu1 = food_recipe[0]
    menu_detail = menu1.splitlines()[1]
    menu_detail = [_.strip() for _ in menu_detail.split(',')]
    temp     = menu_detail[0].split(" ")[1]
    tag_item = re.findall(r"\(.*?\)", temp)[0]
    menu_detail[0].split(" ")[0], temp.replace(tag_item, ""), tag_item
    result = []
    for _ in menu_detail:
        temp     = _.split(" ")[1]
        tag_item = re.findall(r"\(.*?\)", temp)[0]
        result.append([menu1.splitlines()[0], 
                       _.split(" ")[0], 
                       temp.replace(tag_item, ""), 
                       tag_item])
    return result